### IEX Code -------------------------------------------------------
1. Define the stock attributes
2. Define functions: filter attributes, fetch stock data, log events
3. Read company list from the redis datastore
4. Setup schedule
5. Fetch data at scheduled event.

In [1]:
#Imports
import redis
import urllib.request
import json
import numpy as np
import pandas as pd
import schedule
import time
import boto3
from datetime import date, datetime
import traceback
import os

kinesis = boto3.client('kinesis', region_name='us-east-1')

#Mongo
from pymongo import MongoClient


#Connect to Redis-DataStore
REDIS = redis.Redis(host='data_store')

In [ ]:
#Setup Mongo and create the database and collection
client = MongoClient('db-data')
db = client['stock_tweets']
coll_reference = db.iex

In [2]:
########### Set up Feature Flag #####################

   
def get_feature_flag(feature):
    all_flags = pd.read_msgpack(REDIS.get("feature_flags"))
    
    try:
        return all_flags.get_value(feature, 'State')
        
    except:
        return 'Flag Not Found, not a valid feature flag'

In [3]:
############ Define Functions #################################
attributes = ['latestUpdate',
             'companyName',
             'latestPrice',
              'latestVolume',
              'marketCap',
              'open',
              'previousClose',
              'sector',
              'high',
              'low',
              'ytdChange',
              'peRatio',
              'week52High',
              'week52Low'
               ]

#Select interesting stock attributes.
def filter_stock_attributes(data):
    output = {}
    #Choose filter attributes
    for element in attributes:
        output[element]=data[element]
    
    #Convert time
    ctime = output['latestUpdate']/1000
    new_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(ctime))
    output['latestUpdate']= new_time
    
    return output


#Create function to fetch the stock data. This is to prepare for the schedule.

def fetch_stock_data(stocks=[]):
    try:
        #Only pull stock info if the DataFlag is set.
        if int(REDIS.get('Data_On')) == 1:

            for ticker in stocks:
                url = "https://api.iextrading.com/1.0/stock/{}/quote".format(ticker)
                response = urllib.request.urlopen(url)
                str_response = response.read().decode('utf-8')
                obj = json.loads(str_response)
                filtered = filter_stock_attributes(obj)
                #Add counter to count stocks. 
                REDIS.incr('IEX_Stock_Count')
                #<------ Insert into MongoDb ----------->
                if int(get_feature_flag('database_stream_write'))==1:
                        coll_reference.insert_one(filtered)
                
                #<----- Insert to Kinesis Stream ------->
                if int(get_feature_flag('kinesis_stream_write'))==1:
                    response = kinesis.put_record(StreamName="IEX_Stream", Data=json.dumps(filtered), PartitionKey="partitionkey")
                print('---------------------------------')
                #print(response)
                print(filtered)
    except:
        print(traceback.format_exc())
        #Send Error event
        send_event('IEX', 'Error', 'Error occured, check stdout')

#Setup Log

global past_stock_count
past_stock_count = 0




########### Redis Subscription setup ########################
queue = REDIS.pubsub()
#Subscribe to the queues one for the events and one for the log
queue.subscribe('event_queue')
queue.subscribe('log_queue')


#Serialize datetime.
def json_serial(obj):
    """JSON serializer for objects not serializable by default json code"""

    if isinstance(obj, (datetime, date)):
        return obj.isoformat()
    raise TypeError ("Type %s not serializable" % type(obj))
    


#Code to log to the event queue
def send_event(source, kind, message):
    event_time = datetime.now()
    event_time = json_serial(event_time)
    event = {
            "event_time": event_time,
            "source": source,
            "kind" : kind,
            "message" : message
            }
    payload = json.dumps(event)
    REDIS.publish('event_queue', payload)

def send_log(source, current_count, count_diff):
    log_time = datetime.now()
    log_time = json_serial(log_time)
    log = {
            "log_time": log_time,
            "source": source,
            "current_count" : current_count,
            "count_diff" : count_diff
            }
    payload = json.dumps(log)
    REDIS.publish('log_queue', payload)


#Send the log data to the Redis channel.
def log():
    #Need to log: Time, Source, Current Count, Count Diff
    #now = datetime.datetime.now()
    current_stock_count = int(REDIS.get('IEX_Stock_Count'))    
     
    global past_stock_count
    stock_count_diff = current_stock_count - past_stock_count
    past_stock_count = current_stock_count
    
    #Send the log event
    send_log(source='IEX',current_count = current_stock_count, count_diff=stock_count_diff)
    
    print('Logged Data')

In [4]:
#Read Data-Store
companies = json.loads(REDIS.get('companies').decode())
stock_tickers = list(companies.keys())
REDIS.set('IEX_Stock_Count', 0)

#Setup Schedule
schedule.clear()
schedule.every(30).seconds.do(fetch_stock_data, stocks=stock_tickers)
schedule.every(35).seconds.do(log)

#Send Start event
send_event('IEX', 'Activity', 'Data Source Started')

#Execute
while True:
    schedule.run_pending()
    time.sleep(1)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


---------------------------------
{'latestUpdate': '2018-02-28 18:24:30', 'companyName': 'Apple Inc.', 'latestPrice': 179.95, 'latestVolume': 17239008, 'marketCap': 913068639350, 'open': 179.2, 'previousClose': 178.39, 'sector': 'Technology', 'high': 180.615, 'low': 178.51, 'ytdChange': 0.03558574248229418, 'peRatio': 19.56, 'week52High': 180.48, 'week52Low': 136.7}
---------------------------------
{'latestUpdate': '2018-02-28 18:24:30', 'companyName': 'Facebook Inc.', 'latestPrice': 181.62, 'latestVolume': 7636296, 'marketCap': 527606237668, 'open': 182.3, 'previousClose': 181.46, 'sector': 'Technology', 'high': 182.88, 'low': 181.06, 'ytdChange': 0.00022048285745794547, 'peRatio': 29.53, 'week52High': 195.32, 'week52Low': 134.75}
---------------------------------
{'latestUpdate': '2018-02-28 18:23:53', 'companyName': 'Alphabet Inc.', 'latestPrice': 1117.37, 'latestVolume': 941593, 'marketCap': 776904601096, 'open': 1121, 'previousClose': 1118.29, 'sector': 'Technology', 'high': 1127

---------------------------------
{'latestUpdate': '2018-02-28 18:24:34', 'companyName': 'Twitter Inc.', 'latestPrice': 32.345, 'latestVolume': 17106323, 'marketCap': 24288007702, 'open': 31.5, 'previousClose': 31.32, 'sector': 'Technology', 'high': 32.85, 'low': 31.47, 'ytdChange': 0.27784577723378207, 'peRatio': 646.9, 'week52High': 35, 'week52Low': 14.12}
---------------------------------
{'latestUpdate': '2018-02-28 18:24:40', 'companyName': 'General Electric Company', 'latestPrice': 14.195, 'latestVolume': 40291154, 'marketCap': 123249166320, 'open': 14.61, 'previousClose': 14.5, 'sector': 'Industrials', 'high': 14.615, 'low': 14.2, 'ytdChange': -0.19354838709677422, 'peRatio': 13.52, 'week52High': 30.54, 'week52Low': 13.95}
---------------------------------
{'latestUpdate': '2018-02-28 18:24:30', 'companyName': 'UnitedHealth Group Incorporated (DE)', 'latestPrice': 230.19, 'latestVolume': 1351887, 'marketCap': 222746327325, 'open': 234.42, 'previousClose': 232.52, 'sector': 'Heal

---------------------------------
{'latestUpdate': '2018-02-28 18:25:02', 'companyName': 'Pfizer Inc.', 'latestPrice': 37, 'latestVolume': 9798996, 'marketCap': 220255995787, 'open': 37.05, 'previousClose': 36.8, 'sector': 'Healthcare', 'high': 37.28, 'low': 36.75, 'ytdChange': 0.009879253567508219, 'peRatio': 13.96, 'week52High': 39.43, 'week52Low': 31.67}
---------------------------------
{'latestUpdate': '2018-02-28 18:25:11', 'companyName': 'Intel Corporation', 'latestPrice': 50.11, 'latestVolume': 15630545, 'marketCap': 233913480000, 'open': 50.16, 'previousClose': 49.91, 'sector': 'Technology', 'high': 50.32, 'low': 49.32, 'ytdChange': 0.06531483457844173, 'peRatio': 14.44, 'week52High': 50.9, 'week52Low': 33.23}
---------------------------------
{'latestUpdate': '2018-02-28 18:24:48', 'companyName': 'Home Depot Inc. (The)', 'latestPrice': 185.83, 'latestVolume': 2799789, 'marketCap': 217002725869, 'open': 184.26, 'previousClose': 184.98, 'sector': 'Consumer Cyclical', 'high': 18

---------------------------------
{'latestUpdate': '2018-02-28 18:25:25', 'companyName': 'Cisco Systems Inc.', 'latestPrice': 45.71, 'latestVolume': 10695018, 'marketCap': 220208720811, 'open': 45.27, 'previousClose': 45.04, 'sector': 'Technology', 'high': 45.64, 'low': 45.18, 'ytdChange': 0.15903242408646423, 'peRatio': 21.06, 'week52High': 45.71, 'week52Low': 30.36}
---------------------------------
{'latestUpdate': '2018-02-28 18:25:34', 'companyName': 'Exxon Mobil Corporation', 'latestPrice': 77.08, 'latestVolume': 5365430, 'marketCap': 326742120000, 'open': 78.13, 'previousClose': 77.5, 'sector': 'Energy', 'high': 78.25, 'low': 76.94, 'ytdChange': -0.0885569798894508, 'peRatio': 21.77, 'week52High': 89.3, 'week52Low': 73.9}
---------------------------------
{'latestUpdate': '2018-02-28 18:25:44', 'companyName': 'Wells Fargo & Company', 'latestPrice': 59.45, 'latestVolume': 6181354, 'marketCap': 290806608535, 'open': 59.44, 'previousClose': 59.21, 'sector': 'Financial Services', 'h

KeyboardInterrupt: 